# Getting Started with Trends API

This notebook serves as a tutorial for Trends endpoint in the Signal AI API.

Full technical documentation of the Trends endpoint is available here
https://api.signal-ai.com/docs#tag/Trends


## Trends API Overview

Trends API allows the user to get aggregated data from the Signal AI Platform across different dimensions and using different metrics. It allows users to monitor, visualise and understand information such as the following:

- Changes in sentiment for an entity over time
- Sentiment for an entity in relation to a set of key topics
- News coverage and sentiment around an entity in different locations
- Difference in coverage and sentiment between entities over time and in relation to different topics

## 0. Setting things up!

### 0.1 Pre-requisites

Please make sure that you have familiarised yourself with the Signal AI API using the [Getting Started notebook](getting_started.ipynb)

In particular you would need to be able to use the following endpoints:

- Authentication: to be able to access the API
- Discovery: to be able to search for entities and topics of interest


Note: run `pip install -r requirements.txt` to install the dependencies for this notebook

In [2]:
import requests
import os
import pandas as pd
import json

### 0.2 Check if authentication works!

You will need a client_id and client_secret to gain access to the API. The code below will assume they have been set and the environment variables SIGNAL_API_CLIENT_ID and SIGNAL_API_CLIENT_SECRET respectively.

Using your credentials you can request a temporary access token from the API using the url: https://api.signal-ai.com/auth/token


In [3]:
def authenticate(client_id, client_secret, url = "https://api.signal-ai.com"):
    """ obtain a temporary access token using user credentials """
    token_url = f'{url}/auth/token'
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(token_url, data=payload)
    return response.json().get("access_token")

In [4]:
TEMP_ACCESS_TOKEN = authenticate(os.environ['SIGNAL_API_CLIENT_ID'], os.environ['SIGNAL_API_CLIENT_SECRET'])
if TEMP_ACCESS_TOKEN:
    print('Congratulations! You have an access token, it will last for 24 hours before you will need to reauthenticate by repeating this step')
else:
    print('Error: Perhaps the credentials are incorrect?')

Congratulations! You have an access token, it will last for 24 hours before you will need to reauthenticate by repeating this step


## Using Trends API

In [ ]:
Before we start, let us define `request()` function that we can re-use across different example use cases.

In [5]:
def request(json=None):
    """ Make requests using a tempory access token """
    response = requests.post(
        'https://api.signal-ai.com/trends',
        json=json,
        headers={
            "Authorization": f'Bearer {TEMP_ACCESS_TOKEN}',
            "Content-Type": "application/json",
            # since the endpoint is still private, this header is necessary:
            "X-Access": "private"
        },
    )
    return response


Following are some examples of the questions that can be answered using Trends API.

Use case question:

- How has the sentiment around my organisation been changing over time?

Note that you would need to have the ID of the entity as an input. For that please consult the [Getting Started notebook](getting_started.ipynb) to search for entities and obtain the ID of your chosen entity.




**Example**: Sentiment towards Wirecard between the beginning of March 2020 and the end of August 2020 in monthly intervals

In [6]:
entity_id = "eae834b7-5b2f-430e-99ce-770786a971cf" ##  ID for Wirecard

query = {
    # first we define the data to be included in the aggregation. This part of the query has got the same format as the query in `Search` endpoint
    "where": {
        "published-at": {
            "gte": "2020-03-01",
            "lte": "2020-08-31"
        }
    },
    # then we define how the data should be aggregated. Note the `include` field and how we specify the ID(s) of the entities we want the data to be grouped by.
    "aggregations": {
        "group-by": [
            {
                "dimension": "published-at",
                "interval": "month"
            },
            {
                "dimension": "entity",
                "include": [
                    entity_id
                ]
            },
            {
                "dimension": "entity.sentiment"
            }
        ],
        "metrics": ["document-count"]
    }
}

# return the response in JSON format
result = request(json=query).json()


In [ ]:
Let us look at the results: 

In [7]:
print(json.dumps(result, indent=4, sort_keys=True))

{
    "aggregations": {
        "group-by": {
            "dimension": "published-at",
            "groups": [
                {
                    "group-by": {
                        "dimension": "entity",
                        "groups": [
                            {
                                "group-by": {
                                    "dimension": "entity.sentiment",
                                    "groups": [
                                        {
                                            "metrics": {
                                                "document-count": 457
                                            },
                                            "value": "negative"
                                        },
                                        {
                                            "metrics": {
                                                "document-count": 7179
                                            },
                     

In [ ]:
We can also plot and analyse these results:

In [ ]:
# df = pd.DataFrame.from_dict(result)

### 2. Sentiment for an entity in relation to a set of key topics

In [ ]:
Use case question:

- How is sentiment towards organisations in my portfolio trending in relation to a set of key topics? 

In [ ]:
**Example**: Sentiment trending around 7 car manufacturers in January 2021 and in relation to a 6 strategic topics

In [9]:
topics_ids = [
    "fc31abf2-7b11-4ed5-a7d2-35266057c0dd", 
    "c3e9ec2f-e225-4955-9dbf-5480ce3d30fe", 
    "a301b00d-f4ef-49fb-9224-f52386d4955e", 
    "fdcb69a5-8aa6-4067-a29b-f064321e1d7d", 
    "4249987d-5b02-4e51-9c44-019dd8e39742", 
    "8de14dc1-6a93-4d1f-b755-a8d7aa570187"
    ] ## IDs of the following topics: Innovation, Patents & Invention, Corporate Culture, Corporate Governance, Corporate Responsibility, Sustainability

entity_ids = [
    "d6341968-83df-441c-a869-fa7ae9c22c73", 
    "d7f0268d-1322-32b2-83d9-bb6fa9922506", 
    "a9cf01c5-751f-4fe5-a529-12e0d297cb63", 
    "c4ad0758-f3ee-4002-84aa-10849a153d75", 
    "06608104-0136-4371-ad04-be40fcc306a4", 
    "11cab8df-4be1-470f-8f49-8f7f0863ec95", 
    "946b44a8-b767-40ca-8093-e90d6af2a9d4"
    ] ##  IDs of the following car manufacturers: Toyota, Volkswagen, General Motors, Honda, BMW, Tesla, Ford

query = {
    # here we can specify a wider time window, so that we can make a comparison between January 2021 and the month before, in order to analyse the extent of change.
    "where": {
        "published-at": {
            "gte": "2020-12-01",
            "lte": "2021-01-31"
        }
    },
    "aggregations": {
        "group-by": [
            {
                "dimension": "published-at",
                "interval": "month"
            },
            # after grouping by month, we should first group by topics...
            {
                "dimension": "topic",
                "include": topics_ids
            },
            # ...then by entities
            {
                "dimension": "entity",
                "include": entity_ids
            },
            {
                "dimension": "entity.sentiment"
            }
        ],
        "metrics": ["document-count"]
    }
}

result = request(json=query).json()

print(json.dumps(result, indent=4, sort_keys=True))

{
    "aggregations": {
        "group-by": {
            "dimension": "published-at",
            "groups": [
                {
                    "group-by": {
                        "dimension": "topic",
                        "groups": [
                            {
                                "group-by": {
                                    "dimension": "entity",
                                    "groups": [
                                        {
                                            "group-by": {
                                                "dimension": "entity.sentiment",
                                                "groups": [
                                                    {
                                                        "metrics": {
                                                            "document-count": 292
                                                        },
                                                        "value": "n

### 3. News coverage and sentiment around an entity in different locations

In [ ]:
Use case question:

How do news coverage and sentiment around my agency’s clients vary across different countries?

In [ ]:
**Example**: Coverage and sentiment around Uber in February 2021 and in 5 different countries

In [11]:
entity_id = "0c20f04a-94ef-467c-921d-f89f8895c41a" ##  ID for Uber
countries = [
    "United States", 
    "United Kingdom", 
    "Germany", 
    "France", 
    "Spain"
    ]

query = {
    "where": {
        "published-at": {
            "gte": "2021-02-01",
            "lte": "2021-02-28"
        },
        # here we can narrow our search by selecting only the articles where Uber is mentioned
        "entities": {
            "id": {
                "eq": entity_id
            }
        }
    },
    "aggregations": {
        "group-by": [
            {
                "dimension": "source.country",
                "include": countries
            },
            {
                "dimension": "entity",
                "include": [
                    entity_id
                ]
            },
            {
                "dimension": "entity.sentiment"
            }
        ],
        "metrics": ["document-count"]
    }
}

result = request(json=query).json()

print(json.dumps(result, indent=4, sort_keys=True))

{
    "aggregations": {
        "group-by": {
            "dimension": "source.country",
            "groups": [
                {
                    "group-by": {
                        "dimension": "entity",
                        "groups": [
                            {
                                "group-by": {
                                    "dimension": "entity.sentiment",
                                    "groups": [
                                        {
                                            "metrics": {
                                                "document-count": 6646
                                            },
                                            "value": "negative"
                                        },
                                        {
                                            "metrics": {
                                                "document-count": 27493
                                            },
                 

In [ ]:
### 4. Difference in coverage and sentiment between entities over time and in relation to different topics

In [ ]:
Use case question:

How do different organisations that I monitor compare with respect to coverage and sentiment in relation to key topics?

In [ ]:
**Example**: Coverage and sentiment around 4 competitors in the mining industry in January 2021 and in relation to 4 ESG topics

In [12]:
topics_ids = [
    "22297881-a936-4b51-a880-a7152c8ecc68",
    "ff9d1563-570f-4769-bbb5-7f3096beca0a",
    "8de14dc1-6a93-4d1f-b755-a8d7aa570187",
    "f8c74a1b-4f44-4a3f-b9bb-9b7cb221eb3f"
    ] ## IDs of the following topics: Environmental Cleanup, Corporate Controversy,  Sustainability, Chemicals

entity_ids = [
   "72800724-103f-4370-942a-0e715b58ee4a",
   "3d885924-a230-4fac-b17a-d5a1c1151cdf",
   "98eeff9c-6a9a-4d62-801d-eb94b240dd42",
   "592930e0-70fe-45f7-8a1e-ec504158dbf9"
    ] ##  IDs of the following mining companies: Rio Tinto, BHP, Glencore

query = {
    "where": {
        "published-at": {
            "gte": "2021-01-01",
            "lte": "2021-01-31"
        }
    },
    "aggregations": {
        "group-by": [
            # since we are here looking at a particular one-month window, perhaps this time we could use week intervals 
            {
                "dimension": "published-at",
                "interval": "week"
            },
            {
                "dimension": "topic",
                "include": topics_ids
            },
            {
                "dimension": "entity",
                "include": entity_ids
            },
            {
                "dimension": "entity.sentiment"
            }
        ],
        "metrics": ["document-count"]
    }
}

result = request(json=query).json()

print(json.dumps(result, indent=4, sort_keys=True))

{
    "aggregations": {
        "group-by": {
            "dimension": "published-at",
            "groups": [
                {
                    "group-by": {
                        "dimension": "topic",
                        "groups": [
                            {
                                "group-by": {
                                    "dimension": "entity",
                                    "groups": [
                                        {
                                            "group-by": {
                                                "dimension": "entity.sentiment",
                                                "groups": [
                                                    {
                                                        "metrics": {
                                                            "document-count": 0
                                                        },
                                                        "value": "neg